In [2]:
import pandas as pd
import numpy as np
from yahooquery import Ticker

In [157]:
SP_LIST = list(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'])
SP_LIST = [string.replace('.', '-') for string in SP_LIST]


In [163]:
# GLOBAL VARIABLES 

# According to https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/wacc.htm 
# and https://www.linkedin.com/pulse/sp-500-sectors-roic-vs-wacc-through-1q21-david-trainer/

WACC_RATES = {
    'Technology': 0.092,
    'Healthcare': 0.1028,
    'Financial Services': 0.1,
    'Consumer Cyclical': 0.0786,
    'Consumer Defensive': 0.07,
    'Industrials': 0.0791,
    'Energy': 0.0867,
    'Utilities': 0.068,
    'Basic Materials': 0.08,
    'Real Estate': 0.0986,
    'Communication Services': 0.073,
}

SECTORS = list(WACC_RATES.keys())

AVGPERATIOS = {'Technology': 49.23658881690141,
 'Healthcare': 56.14869727118644,
 'Financial Services': 19.586146538461545,
 'Consumer Cyclical': 24.644157226415093,
 'Consumer Defensive': 31.269938114285708,
 'Industrials': 30.180891085714283,
 'Energy': 13.010462826086957,
 'Utilities': 19.340905928571427,
 'Basic Materials': 26.606018799999998,
 'Real Estate': 50.20442143333334,
 'Communication Services': 30.231775736842103}


In [ ]:
def calculate_dcf_intrinsic_value(stock, sector):

    ticker = Ticker(stock)

    # Assuming constant growth rate for simplicity
    growth_rate = 0.04  # You can adjust this according to your analysis

    # Discount rate
    discount_rate = WACC_RATES[sector]  

    # Get last year free cash flow
    fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'].iloc[-1]

    # Get the number of shares outstanding
    shares_outstanding = ticker.key_stats[stock]['sharesOutstanding']

    # Calculate terminal value using perpetual growth formula
    terminal_value = (fcf * (1 + growth_rate)) / (discount_rate - growth_rate)

    # Discount each year's cash flow
    discounted_cash_flows = [fcf / (1 + discount_rate) ** i for i in range(1, 6)]  # Discounting cash flows for 5 years

    # Add terminal value
    discounted_cash_flows.append(terminal_value / (1 + discount_rate) ** 5)

    # Calculate intrinsic value
    intrinsic_value = np.sum(discounted_cash_flows) / shares_outstanding

    return intrinsic_value

In [ ]:
def get_peratio(stock):
    ticker = Ticker(stock)
    try:
        peratio = ticker.valuation_measures['PeRatio'].iloc[-2] # current P/E for the stock
    except:
        peratio = None
    
    return peratio

In [173]:
# Calculate pe ratio sector by sector
PERATIOS = {}
for sector in SECTORS:
    PERATIOS[sector] = []

for stock in SP_LIST:
    ticker = Ticker(stock)
    try:
        peratio = ticker.valuation_measures['PeRatio'].iloc[-2] # current P/E for the stock
    except:
        peratio = None
    
    if peratio != None:
        sector = ticker.asset_profile[stock]['sector']
        if isinstance(sector, str) and sector in PERATIOS:
            PERATIOS[sector].append(peratio)

print(PERATIOS)

/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/855217350.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock
/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/855217350.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock
/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/855217350.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treate

{'Technology': [35.294717, 45.933164, 397.471698, 31.295455, 35.533762, 34.235714, 58.670157, 26.303266, 24.961176, 43.237082, 59.937947, 40.933899, 35.369338, 81.910995, 30.517284, 14.948328, 18.213777, 48.25, 183.885714, 40.707792, 43.910765, 24.850065, 72.001677, 82.152941, 20.4677, 30.248996, 21.609091, 48.856164, 34.662551, 20.85693, 42.325812, 9.714932, 12.944828, 8.865103, 24.068712, 111.3, 66.460674, 26.052721, 34.133072, 39.410526, 28.887868, 36.227342, 37.865793, 90.159722, 20.516355, nan, 36.355335, 83.531963, 33.990937, 23.872437, 74.378877, 23.481308, 15.879346, 31.013812, 43.168731, 29.911565, 92.353535, nan, 23.957204, 42.384615, 72.32619, nan, 88.011876, 18.528369, 63.572848, 13.442015, 23.007572, 39.102564, 24.30976, 49.944, 108.21134, 90.390805, 24.218987, nan, 50.29021], 'Healthcare': [36.607362, 66.566176, 34.980861, 52.301205, 22.146517, nan, 54.036697, 53.726619, 27.29611, 63.009346, 13.829016, 43.896825, nan, 25.56427, 15.674747, 28.572668, 19.541116, 69.900515, 

/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/855217350.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


In [185]:
AVGPERATIOS = {}
for sector in SECTORS:
    peratios = [x for x in PERATIOS[sector] if str(x) != 'nan']
    AVGPERATIOS[sector] = sum(peratios)/len(peratios)
AVGPERATIOS

71
59
65
53
35
70
23
28
20
30
19


{'Technology': 49.23658881690141,
 'Healthcare': 56.14869727118644,
 'Financial Services': 19.586146538461545,
 'Consumer Cyclical': 24.644157226415093,
 'Consumer Defensive': 31.269938114285708,
 'Industrials': 30.180891085714283,
 'Energy': 13.010462826086957,
 'Utilities': 19.340905928571427,
 'Basic Materials': 26.606018799999998,
 'Real Estate': 50.20442143333334,
 'Communication Services': 30.231775736842103}

In [ ]:


# 71
# 59
# 65
# 53
# 35
# 70
# 23
# 28
# 20
# 30
# 19

In [174]:
# Calculating intrinsic value using P/E ratio
# Based on https://www.cmcmarkets.com/en/trading-guides/intrinsic-value#:~:text=EPS%20x%20(1%20%2B%20r)%20x%20P%2FE&text=r%20is%20expected%20growth%20rate,0.15)%20x%2029%20%3D%20%24170.
def calculate_pe_intrinsic_value(stock):
    ticker = Ticker(stock)
    eps = ticker.key_stats[stock]['trailingEps'] # amount of earning over the last year
    r = ticker.earnings_trend[stock]['trend'][-3]['growth'] # expected growth rate of earnings
    try:
        peratio = ticker.valuation_measures['PeRatio'].iloc[-2] # current P/E for the stock
    except:
        return None
    pe_intrinsic_value = eps * (1 + r) * peratio

    return pe_intrinsic_value

In [ ]:
# calculate dcf for all stocks
# take the naans out of the list
# calculate the average

In [154]:
upside_list = []
dcf_upside_list = []
pe_upside_list = []
count = 0

for stock in SP_LIST[:25]:
    # STOCK VARIABLES

    ticker = Ticker(stock)
    CURRENT_PRICE = ticker.financial_data[stock]['currentPrice']
    sector = ticker.asset_profile[stock]['sector']

    if isinstance(sector, str):
        dcf_intrinsic_value = calculate_dcf_intrinsic_value(stock, sector)
        pe_intrinsic_value = calculate_pe_intrinsic_value(stock)

        if dcf_intrinsic_value != None and pe_intrinsic_value != None:
            dcf_upside_list.append(dcf_intrinsic_value/CURRENT_PRICE - 1)
            pe_upside_list.append(pe_intrinsic_value/CURRENT_PRICE - 1)

            print(f"Upside for DCF intrinsic value of {stock}: {(dcf_intrinsic_value/CURRENT_PRICE - 1):.2f}")
            print(f"Upside for PE intrinsic value of {stock}: {(pe_intrinsic_value/CURRENT_PRICE - 1):.2f}")

            intrinsic_value = (pe_intrinsic_value + dcf_intrinsic_value)/2
            upside = intrinsic_value/CURRENT_PRICE - 1
            upside_list.append(upside)
        else:
            count += 1
    else:
        count += 1

average_upside = sum(upside_list)/len(upside_list)
print("Average upside is:", average_upside)

print(dcf_upside_list)
print(pe_upside_list)
print(count)

/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/3641086962.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'][-1]
/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/2778714578.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


Upside for DCF intrinsic value of MMM: 1.19
Upside for PE intrinsic value of MMM: nan


/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/3641086962.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'][-1]
/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/2778714578.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


Upside for DCF intrinsic value of AOS: 0.26
Upside for PE intrinsic value of AOS: 0.04


/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/3641086962.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'][-1]
/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/2778714578.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


...

/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/3641086962.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'][-1]
/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/2778714578.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


Upside for DCF intrinsic value of AMCR: 0.21
Upside for PE intrinsic value of AMCR: 0.04


/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/3641086962.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'][-1]


Upside for DCF intrinsic value of AEE: -2.91
Upside for PE intrinsic value of AEE: 0.06
Average upside is: nan
[1.1945843413466033, 0.26122432700751563, -0.6644486945590473, -0.03743526809307074, -0.3788650088099267, -0.5379172800995444, -0.9450255764188177, -13.606712590140468, -0.022251992245611474, -0.5311905322801633, -1.7508613409474747, 0.21780357839051345, -0.39417186870540355, -2.647319681932999, 0.1181897320390295, -0.6374743539829295, -0.008185268403596013, 1.1290516240974773, 1.3014286813946967, 1.3763138480017587, 1.787131221126642, -0.6059700257652696, 0.2126872082454927, -2.913558668438936]
[nan, 0.040525101104096795, 0.09845700240042987, 0.09396056759085125, 0.08367762085373842, 0.10718057174361562, 0.5058532496420669, 0.08802474442574271, 0.06930802537586578, 0.0860490179413298, 0.0777825235257168, 0.1402963239535282, 0.06625054138537845, 1.2882579522602313, 0.05132178667869525, 0.10680658148648647, 0.04241536707520854, 0.07324704224419065, 0.12247345965173384, 0.129154

/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/2778714578.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


In [ ]:
Average upside is: nan
[-0.19622320047089725, -0.10941772143781348, -0.2276075550669493, -0.30173320456538777, -0.8794319153673094, 0.10898628098914043, -0.10684553247727224, 1.3338159218109542, 0.6240939145413049, 0.8526892186296973, -0.34844321783449383, -0.5712405147292665, -0.17812019937359713, 0.17248124719181535, -0.7512818136588378, -0.4606386670600623, -0.5510910041844289, -0.374592867809425, 0.32300379176183047, 0.7111382071170684, 0.9551250709863377, -0.7618042166759583, 0.6340886917930442, -0.5470427771222135, -0.48074640203359387, -0.44208943045795557, -0.38650520254575915, 0.6311450334693633, -11.204045629012814, -0.7588294222091603, 0.282924817306182, -0.4584663898423966, 0.3845958177986506, 4.45186930330843, 0.18689525010312802, 0.5177378137367881, 0.2775593889283263, -0.11529021579006016, -1.4399186239445, 0.27848158828028113, -0.2271481120697254, -0.9081779956521111, -0.2710900660148353, 6.4358017095415985, 0.006812535158019273, 1.3693630963641987, -1.904184266719927, 12.611387346115915]
[0.09153110339119741, 0.08614731205869397, 0.09132054830114034, 0.11873757496252435, 0.2903077107987182, 0.23549208391047682, 0.10763453508698961, -0.10644689282514719, -0.05702542383745268, 0.15963732698058886, nan, 0.09858092249541306, 0.04294339793973512, 0.0956481133075957, 0.12010294350324902, 0.12719395747994144, 0.13615562221344235, 0.22040326293668988, 0.1556353392592471, 0.10919273359355652, -0.0784380092766429, 0.12631760910144507, 0.04295386021058545, 0.1547728678810385, 0.09379116757930706, 0.07862178091089111, 0.0832692326923079, 0.09680436942379833, 0.028173840966591035, 0.15109106103949177, 1.0730827981751019, 0.04301152742961323, 0.12097225955016122, 0.0018835312295677475, 0.06756521817454031, 0.18082768418968498, nan, 0.054565251718555974, nan, 0.033819012842845275, 0.08401307764517729, 0.24842557405556764, 0.09391080953370379, 0.18844125840674075, 0.07153052426461315, 0.09187030054669831, 0.0736206128194985, -0.022557189980329073]
2

In [171]:
dcf_upside_list_one = [0.26122432700751563, -0.6644486945590473, -0.03743526809307074, -0.3788650088099267, -0.5379172800995444, -0.9450255764188177, -13.606712590140468, -0.022251992245611474, -0.5311905322801633, -1.7508613409474747, 0.21780357839051345, -0.39417186870540355, -2.647319681932999, 0.1181897320390295, -0.6374743539829295, -0.008185268403596013, 1.1290516240974773, 1.3014286813946967, 1.3763138480017587, 1.787131221126642, -0.6059700257652696, 0.2126872082454927, -2.913558668438936]
pe_upside_list_one = [0.040525101104096795, 0.09845700240042987, 0.09396056759085125, 0.08367762085373842, 0.10718057174361562, 0.5058532496420669, 0.08802474442574271, 0.06930802537586578, 0.0860490179413298, 0.0777825235257168, 0.1402963239535282, 0.06625054138537845, 1.2882579522602313, 0.05132178667869525, 0.10680658148648647, 0.04241536707520854, 0.07324704224419065, 0.12247345965173384, 0.1291543309523806, 0.035501818544404484, 0.2333272834034612, 0.04417098394904473, 0.0635818327881772]


dcf_upside_list_two = [1.7441506487452982, -2.739075556922396, 0.5294287181907917, 0.7856809277738592, -0.5438016086856647, -2.1486632542718898, 0.5823172331968565, -0.15447996083686144, -0.2996474724059034, -0.4593751631268721, -0.39410955862915387, -0.6076213310722935, -0.26235210387043795, 0.5601941539218307, -0.3608452684418779, -0.2799816408409276, 0.011804259432891318, 1.5270531908572784, 0.6760916284753633, -0.6260511864319565, -0.5062678859215228, 0.46421390992696465, 3.3998923430720547, 0.1568909332003201, -0.37536317850514855]
pe_upside_list_two = [0.3062443312939598, 0.061722720167419975, 0.014850651333631726, 0.14498858748394006, 0.03318045883309706, 0.09953662260854923, 0.08951047660064426, 0.08473314786298047, 0.09571934142050864, 0.10361130990390666, 0.21952642242937848, 0.09566954718543186, 0.09913871039616029, 0.325461725155483, 0.09183923922698223, 0.15696301108169863, 0.2398439299542976, 0.07771106582552179, -0.16653922176129032, 0.1333751034969981, 0.11175413541695, 0.09575343698517136, 0.04893721839882703, 0.06339995326170067, 0.11501251838603088]


dcf_upside_list_three = [-0.19622320047089725, -0.10941772143781348, -0.2276075550669493, -0.30173320456538777, -0.8794319153673094, 0.10898628098914043, -0.10684553247727224, 1.3338159218109542, 0.6240939145413049, 0.8526892186296973, -0.5712405147292665, -0.17812019937359713, 0.17248124719181535, -0.7512818136588378, -0.4606386670600623, -0.5510910041844289, -0.374592867809425, 0.32300379176183047, 0.7111382071170684, 0.9551250709863377, -0.7618042166759583, 0.6340886917930442, -0.5470427771222135, -0.48074640203359387, -0.44208943045795557, -0.38650520254575915, 0.6311450334693633, -11.204045629012814, -0.7588294222091603, 0.282924817306182, -0.4584663898423966, 0.3845958177986506, 4.45186930330843, 0.18689525010312802, 0.5177378137367881, -0.11529021579006016, 0.27848158828028113, -0.2271481120697254, -0.9081779956521111, -0.2710900660148353, 6.4358017095415985, 0.006812535158019273, 1.3693630963641987, -1.904184266719927, 12.611387346115915]
pe_upside_list_three = [0.09153110339119741, 0.08614731205869397, 0.09132054830114034, 0.11873757496252435, 0.2903077107987182, 0.23549208391047682, 0.10763453508698961, -0.10644689282514719, -0.05702542383745268, 0.15963732698058886, 0.09858092249541306, 0.04294339793973512, 0.0956481133075957, 0.12010294350324902, 0.12719395747994144, 0.13615562221344235, 0.22040326293668988, 0.1556353392592471, 0.10919273359355652, -0.0784380092766429, 0.12631760910144507, 0.04295386021058545, 0.1547728678810385, 0.09379116757930706, 0.07862178091089111, 0.0832692326923079, 0.09680436942379833, 0.028173840966591035, 0.15109106103949177, 1.0730827981751019, 0.04301152742961323, 0.12097225955016122, 0.0018835312295677475, 0.06756521817454031, 0.18082768418968498, 0.054565251718555974, 0.033819012842845275, 0.08401307764517729, 0.24842557405556764, 0.09391080953370379, 0.18844125840674075, 0.07153052426461315, 0.09187030054669831, 0.0736206128194985, -0.022557189980329073]

dcf_upside_list = dcf_upside_list_one + dcf_upside_list_two + dcf_upside_list_three
pe_upside_list = pe_upside_list_one + pe_upside_list_two + pe_upside_list_three

print (dcf_upside_list)
dcf_average_upside = sum(dcf_upside_list)/len(dcf_upside_list)
print("Average dcf_upside_list is:", dcf_average_upside)

pe_average_upside = sum(pe_upside_list)/len(pe_upside_list)
print("Average pe_upside_list is:", pe_average_upside)

def calculate_variance(numbers):
    # Step 1: Calculate the mean
    mean = sum(numbers) / len(numbers)
    
    # Step 2: Calculate the squared differences
    squared_diff = [(x - mean) ** 2 for x in numbers]
    
    # Step 3: Calculate the mean of the squared differences (variance)
    variance = sum(squared_diff) / len(numbers)
    
    return variance

print('dcf_upside_list variance:', calculate_variance(dcf_upside_list))
print('pe_upside_list variance:', calculate_variance(pe_upside_list))

[0.26122432700751563, -0.6644486945590473, -0.03743526809307074, -0.3788650088099267, -0.5379172800995444, -0.9450255764188177, -13.606712590140468, -0.022251992245611474, -0.5311905322801633, -1.7508613409474747, 0.21780357839051345, -0.39417186870540355, -2.647319681932999, 0.1181897320390295, -0.6374743539829295, -0.008185268403596013, 1.1290516240974773, 1.3014286813946967, 1.3763138480017587, 1.787131221126642, -0.6059700257652696, 0.2126872082454927, -2.913558668438936, 1.7441506487452982, -2.739075556922396, 0.5294287181907917, 0.7856809277738592, -0.5438016086856647, -2.1486632542718898, 0.5823172331968565, -0.15447996083686144, -0.2996474724059034, -0.4593751631268721, -0.39410955862915387, -0.6076213310722935, -0.26235210387043795, 0.5601941539218307, -0.3608452684418779, -0.2799816408409276, 0.011804259432891318, 1.5270531908572784, 0.6760916284753633, -0.6260511864319565, -0.5062678859215228, 0.46421390992696465, 3.3998923430720547, 0.1568909332003201, -0.37536317850514855,

In [172]:
upside_list = []
dcf_upside_list = []
pe_upside_list = []
count = 0

for stock in SP_LIST[100:150]:
    # STOCK VARIABLES

    ticker = Ticker(stock)
    CURRENT_PRICE = ticker.financial_data[stock]['currentPrice']
    sector = ticker.asset_profile[stock]['sector']
    if isinstance(sector, str):
        dcf_intrinsic_value = calculate_dcf_intrinsic_value(stock, sector)
        pe_intrinsic_value = calculate_pe_intrinsic_value(stock)

        if dcf_intrinsic_value != None and pe_intrinsic_value != None:
            dcf_upside_list.append(dcf_intrinsic_value/CURRENT_PRICE - 1)
            pe_upside_list.append(pe_intrinsic_value/CURRENT_PRICE - 1)

            print(f"Upside for DCF intrinsic value of {stock}: {(dcf_intrinsic_value/CURRENT_PRICE - 1):.2f}")
            print(f"Upside for PE intrinsic value of {stock}: {(pe_intrinsic_value/CURRENT_PRICE - 1):.2f}")

            intrinsic_value = (pe_intrinsic_value + dcf_intrinsic_value)/2
            upside = intrinsic_value/CURRENT_PRICE - 1
            upside_list.append(upside)
        else:
            count += 1
    else:
        count += 1

average_upside = sum(upside_list)/len(upside_list)
print("Average upside is:", average_upside)

print(dcf_upside_list)
print(pe_upside_list)
print(count)

/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/3641086962.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'][-1]
/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/2778714578.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


Upside for DCF intrinsic value of CHRW: 0.69
Upside for PE intrinsic value of CHRW: 0.23


/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/3641086962.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'][-1]
/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/2778714578.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


Upside for DCF intrinsic value of CRL: -0.63
Upside for PE intrinsic value of CRL: 0.10


/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/3641086962.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'][-1]
/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/2778714578.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


...

/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/3641086962.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'][-1]
/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/2778714578.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


Upside for DCF intrinsic value of DLR: -0.49
Upside for PE intrinsic value of DLR: 0.06


/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/3641086962.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fcf = ticker.cash_flow(trailing=False)['FreeCashFlow'][-1]


Upside for DCF intrinsic value of DFS: 2.95
Upside for PE intrinsic value of DFS: 0.24
Average upside is: nan
[0.692988427865894, -0.6336187666685633, 1.2729043712282353, 1.234858398704358, 0.32746800125258746, -0.6276559749033661, 0.8208904352397504, -0.08367397995949633, 0.4222028995712612, 0.6117785666480173, -0.5589310426522854, 0.6103689032877897, -11.553065355834066, 1.6107146274485036, 0.40432653151675413, -0.3605657379891499, -1.4621867148922822, 0.09576305376493366, -0.12566345339227125, 0.20745397346000805, 1.0551414118351983, 1.3163107132649023, 0.3731349560144468, 0.22367620840116298, -3.3213209925001004, 0.06301420775486899, -5.190148993367804, -0.8525263314881539, -0.6458438073399011, -0.6297704349093087, -0.3341668328899371, -0.8016494219280097, -0.4424051958913252, 0.4757579131669907, -0.4769071633014703, -0.024297179178843997, 0.6169559625418792, 0.5471649969838206, -0.5719995225273726, 0.03331501724580699, 0.7724139839586859, -0.8305764023677058, -0.125088213382325, -

/var/folders/dj/d6jjsbn56jbg6j4z1r0_rgzr0000gn/T/ipykernel_59263/2778714578.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peratio = ticker.valuation_measures['PeRatio'][-2] # current P/E for the stock


INCOME

The primary objective is to obtain an above average and ongoing secure income stream
from dependable sources, with some emphasis on protection of principal. Equity focus is
not just on the current dividend yield, but also on the potential for dividend growth as an
offset against inflation

STRATEGY

- dividend yield higher than the current yield on the S&P 500
- the potential for dividend growth in excess of the inflation rate (as well as above average dividend growth)
- whether the stock will go down or not (if it will do down, will the dividend growth offset it?)

In [ ]:

# Ticker for the ETF for S&P500
'SPY'

"dividendRate": 3.28,
    "dividendYield": 0.0086,
    "exDividendDate": "2020-05-08 00:00:00",
    "payoutRatio": 0.2408,
    "fiveYearAvgDividendYield": 1.57,
    "beta": 1.182072,

In [12]:
Ticker('SPY').dividend_history(start = '2019-01-01')

/Users/tetsu/miniconda3/envs/yquery_env/lib/python3.12/site-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
/Users/tetsu/miniconda3/envs/yquery_env/lib/python3.12/site-packages/yahooquery/ticker.py:1333: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["dividends"].fillna(0, inplace=True)


dividends
symbol date                 
SPY    2019-03-15      1.233
       2019-06-21      1.432
       2019-09-20      1.384
       2019-12-20      1.570
       2020-03-20      1.406
       2020-06-19      1.366
       2020-09-18      1.339
       2020-12-18      1.580
       2021-03-19      1.278
       2021-06-18      1.376
       2021-09-17      1.428
       2021-12-17      1.633
       2022-03-18      1.366
       2022-06-17      1.577
       2022-09-16      1.596
       2022-12-16      1.781
       2023-03-17      1.506
       2023-06-16      1.638
       2023-09-15      1.583
       2023-12-15      1.906

In [40]:
import warnings

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [63]:
dividend_history = Ticker('aapl').dividend_history(start = '2014-01-01')['dividends']
dividend_history
df = pd.DataFrame(dividend_history.items(), columns=['date', 'dividends'])
df = df.explode('date').reset_index(drop=True)
value_to_drop = 'aapl'
df = df[df['date'] != value_to_drop]
# df.rename(columns={'date': 'date', 0: 'dividends'}, inplace=True)


df

,date,dividends
1,2014-02-06,0.108929
3,2014-05-08,0.117500
5,2014-08-07,0.117500
7,2014-11-06,0.117500
9,2015-02-05,0.117500
11,2015-05-07,0.130000
13,2015-08-06,0.130000
15,2015-11-05,0.130000
17,2016-02-04,0.130000
19,2016-05-05,0.142500


In [154]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def predict_dividend_growth(df, future_years):
    
    # Convert date column to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Extract year from date
    df['year'] = df['date'].dt.year

    # Calculate dividend growth rate
    df['dividend_growth'] = df['dividends'].pct_change() * 100
    
    # Prepare features and target variable
    X = df[['year']].values

    y = df['dividend_growth'].values
    
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Remove rows with missing values
    # Find indices of NaN values in both arrays
    nan_indices_ytrain = np.isnan(y_train).flatten()
    nan_indices_ytest = np.isnan(y_test).flatten()

    X_train = X_train[~nan_indices_ytrain]
    X_test = X_test[~nan_indices_ytest]
    y_train = y_train[~nan_indices_ytrain]
    y_test = y_test[~nan_indices_ytest]

    # Initialize and train the linear regression model
    model = LinearRegression()
    if len(X_train.shape) == 1:
        X_train = X_train.reshape(-1, 1)
    if len(X_test.shape) == 1:
        X_test = X_test.reshape(-1, 1)
    model.fit(X_train, y_train)
    
    # Predict future dividend growth
    future_years_array = [[year] for year in range(df['year'].max() + 1, df['year'].max() + future_years + 1)]
    future_dividend_growth = model.predict(future_years_array)

    # Assuming you have already trained the model and have predictions
    y_pred = model.predict(X_test)

    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)

    # Calculate RMSE
    rmse = np.sqrt(mse)

    # Calculate R-squared
    r2 = r2_score(y_test, y_pred)

    print("Mean Squared Error (MSE):", mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("R-squared (R2):", r2)
        
    return future_dividend_growth

In [159]:
STARTDATE_LIST = ['2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01'
                  '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01', '2024-01-01']
ticker = 'aapl'
for start_date in STARTDATE_LIST:
    dividend = Ticker(ticker).dividend_history(start = start_date)
    print(dividend)
    if dividend.shape[0] != 0:
        dividend_history = dividend['dividends']
        dividend_history = pd.DataFrame(dividend_history.items(), columns=['date', 'dividends'])
        dividend_history = dividend_history.explode('date').reset_index(drop=True)
        dividend_history = dividend_history[dividend_history['date'] != ticker]
        dividend_history = dividend_history.reset_index(drop=True)
        print(dividend_history['dividends'])
        
        # # Convert 'date' column to datetime
        # dividend_history['date'] = pd.to_datetime(dividend_history['date'])
        # # Group by year and sum up dividends
        # dividend_history = dividend_history.groupby(dividend_history['date'].dt.year)['dividends'].sum()
        # dividend_history = dividend_history.drop(dividend_history.index[-1])
        # dividend_history = dividend_history.rename_axis('year').reset_index(name='dividends')
        # print(dividend_history)
        break

future_years = 5
predicted_dividend_growth = predict_dividend_growth(dividend_history, future_years)
print("Predicted dividend growth for the next", future_years, "years:", predicted_dividend_growth)

predicted_dividends = []
recent_dividend = dividend_history['dividends'].iloc[-1]

def calculate_future_dividends(last_dividend, predicted_dividend_growth):
    future_dividends = []
    growth_factor = 1  # Start with a growth factor of 1

    # Calculate future dividends for each year based on the predicted growth rates
    for growth_rate in predicted_dividend_growth:
        growth_factor *= (1 + growth_rate / 100)  # Convert percentage to factor
        future_dividend = last_dividend * growth_factor
        future_dividends.append(future_dividend)

    return future_dividends


future_dividends = calculate_future_dividends(recent_dividend, predicted_dividend_growth)
print("Projected dividend for the next 5 years:", future_dividends)

                   dividends
symbol date                 
aapl   2014-02-06   0.108929
       2014-05-08   0.117500
       2014-08-07   0.117500
       2014-11-06   0.117500
       2015-02-05   0.117500
       2015-05-07   0.130000
       2015-08-06   0.130000
       2015-11-05   0.130000
       2016-02-04   0.130000
       2016-05-05   0.142500
       2016-08-04   0.142500
       2016-11-03   0.142500
       2017-02-09   0.142500
       2017-05-11   0.157500
       2017-08-10   0.157500
       2017-11-10   0.157500
       2018-02-09   0.157500
       2018-05-11   0.182500
       2018-08-10   0.182500
       2018-11-08   0.182500
       2019-02-08   0.182500
       2019-05-10   0.192500
       2019-08-09   0.192500
       2019-11-07   0.192500
       2020-02-07   0.192500
       2020-05-08   0.205000
       2020-08-07   0.205000
       2020-11-06   0.205000
       2021-02-05   0.205000
       2021-05-07   0.220000
       2021-08-06   0.220000
       2021-11-05   0.220000
       2022-02

In [215]:
def predict_growth(data, future_years):
    
    X = np.arange(len(data)).reshape(-1, 1)
    y = data

    # Create and fit the linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict growth for the next 5 time points
    future_time_points = np.arange(len(data), len(data) + future_years).reshape(-1, 1)
    predicted_growth = model.predict(future_time_points)

    return list(predicted_growth)

In [235]:
ticker = 'GTMEF'

In [236]:
STARTDATE_LIST = ['2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01'
                  '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01', '2024-01-01']

found = 0
for start_date in STARTDATE_LIST:
    dividend = Ticker(ticker).dividend_history(start = start_date)
    if dividend.shape[0] != 0:
        dividend_history = dividend['dividends']
        dividend_history = pd.DataFrame(dividend_history.items(), columns=['date', 'dividends'])
        dividend_history = dividend_history.explode('date').reset_index(drop=True)
        dividend_history = dividend_history[dividend_history['date'] != ticker]
        dividend_history = dividend_history.reset_index(drop=True)

        # Convert 'date' column to datetime
        dividend_history['date'] = pd.to_datetime(dividend_history['date'])
        # Group by year and sum up dividends
        dividend_history = dividend_history.groupby(dividend_history['date'].dt.year)['dividends'].sum()
        dividend_history = dividend_history.drop(dividend_history.index[-1])
        dividend_history = dividend_history.rename_axis('year').reset_index(name='dividends')
        data = dividend_history['dividends']
        found = 1
        break

if found == 0:
    return None

dividend_pred_fiveyrs = predict_growth(data, 5)

In [237]:
STARTDATE_LIST = ['2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01'
                  '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01', '2024-01-01']

found = 0
for start_date in STARTDATE_LIST:
    price_history_array = np.array(Ticker(ticker).history(start = start_date, interval = '1mo')['close'])
    if len(price_history_array) != 0:
        found = 1
        break
if found == 0:
    return None

# Set pred num to 60 because we want to predict 5 years (60months) ahead
price_pred = predict_growth(price_history_array, 60)
index_list = [11, 23, 35, 47, 59]

price_pred_fiveyrs = []
for index in index_list:
    price_pred_fiveyrs.append(price_pred[index])


In [238]:

# Perform element-wise division
dividend_yield_pred_fiveyrs = [a / b for a, b in zip(dividend_pred_fiveyrs, price_pred_fiveyrs)]

dividend_yield_pred_fiveyrs



[0.045819791035905014,
 0.046630809748121685,
 0.047450975659615216,
 0.048280444400173615,
 0.049119375150220294]

In [ ]:
msft
[0.007631832397067421,
 0.007411382221668161,
 0.007225734223409855,
 0.007067250068695685,
 0.006930374308856264]

aapl
[0.005304651511157489,
 0.005116046373053951,
 0.004957369141846832,
 0.004822019997265974,
 0.0047052067981185695]

SPY
[0.013279079089764329,
 0.013067319410250968,
 0.012878794231973427,
 0.01270987845778743,
 0.012557663836589825]

In [ ]:
STARTDATE_LIST = ['2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01'
                  '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01', '2024-01-01']
ticker = 'aapl'
for start_date in STARTDATE_LIST:
    dividend = Ticker(ticker).history(start = start_date, interval = '1d')['dividends']
    print(dividend)
    if dividend.shape[0] != 0:
        dividend = pd.DataFrame(dividend.items(), columns=['date', 'dividends'])
        dividend = dividend.explode('date').reset_index(drop=True)
        dividend = dividend[dividend['date'] != ticker]
        dividend = dividend.reset_index(drop=True)
        # data = dividend_history['dividends'].values
        print(dividend)
        # Convert 'date' column to datetime
        dividend['date'] = pd.to_datetime(dividend['date'])

        # Group by year and sum up the dividends
        dividend['year'] = dividend['date'].dt.year
        print(dividend)
        dividend = dividend.groupby(dividend['date'].dt.year)['dividends'].sum()
        dividend = dividend.drop(dividend.index[-1])
        dividend = dividend.rename_axis('year').reset_index(name='dividends')
        print('result:',dividend)
        break

dividend_pred_fiveyrs = predict_growth(data, 5)

In [3]:
dir(Ticker('^GSPC'))

['CHUNK',
 'CORPORATE_EVENTS',
 'FUNDAMENTALS_OPTIONS',
 'FUNDAMENTALS_TIME_ARGS',
 'INTERVALS',
 'MODULES',
 'PERIODS',
 '_CONFIG',
 '_FUND_DETAILS',
 '_MODULES_DICT',
 '_STYLE_BOX',
 '_VIZ_CONFIG',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adjust_ohlc',
 '_async_requests',
 '_chunk_symbols',
 '_construct_data',
 '_construct_params',
 '_construct_urls',
 '_country',
 '_country_params',
 '_financials',
 '_financials_dataframes',
 '_format_data',
 '_fund_holdings',
 '_get_data',
 '_get_symbol',
 '_historical_data_to_dataframe',
 '_history_1m',
 '_option_dataframe',
 '_quote_summary',
 '_quote_summary_dataframe',
 '_setup_url',
 '_symbols',
 '_sy

In [242]:
predicted_prices = [105, 107, 120, 110, 113]
predicted_market_prices = [204, 207, 200, 210, 213]
current_price = 100
current_market_price = 200
beta = 1.5

predicted_market_prices[i+1] - 

def calculate_interval(predicted_prices, predicted_market_prices, current_price, current_market_price, beta):
    intervals = []
    for i in range(len(predicted_prices)):
        expected_price = current_price + beta * (predicted_market_prices[i] - current_market_price)
        lower_bound = expected_price - predicted_prices[i]
        upper_bound = expected_price + predicted_prices[i]
        intervals.append((lower_bound, upper_bound))
    return intervals

intervals = calculate_interval(predicted_prices, predicted_market_prices, current_price, current_market_price, beta)

for i, interval in enumerate(intervals):
    print(f"Interval for prediction {i+1}: {interval}")


    Expected Price=Current Price+β×(Predicted Market Price−Current Market Price)

Interval for prediction 1: (1.0, 211.0)
Interval for prediction 2: (3.5, 217.5)
Interval for prediction 3: (-20.0, 220.0)
Interval for prediction 4: (5.0, 225.0)
Interval for prediction 5: (6.5, 232.5)


In [241]:
# Given data
predicted_prices = [105, 107, 120, 110, 113]
predicted_market_prices = [204, 207, 200, 210, 213]  # Predicted market prices
current_price = 100
current_market_price = 200  # Assuming current price of the stock
beta = 1.5  # Beta of the stock

# Calculate expected returns for each year
expected_returns = [(predicted_price / current_price) - 1 for predicted_price in predicted_prices]

# Calculate expected market returns for each year
expected_market_returns = [(predicted_market_price / current_market_price) - 1 for predicted_market_price in predicted_market_prices]

# Adjust returns based on beta
adjusted_returns = [expected_market_return * beta for expected_market_return in expected_market_returns]

# Calculate bottom end and top end prices for each year
bottom_end_prices = [(1 + adjusted_return) * current_price for adjusted_return in adjusted_returns]
top_end_prices = [(1 + adjusted_return) * predicted_price for predicted_price, adjusted_return in zip(predicted_prices, adjusted_returns)]

# Print the results
for year, bottom_price, top_price in zip(range(1, len(predicted_prices) + 1), bottom_end_prices, top_end_prices):
    print(f"Year {year}: Bottom End Price = {bottom_price:.2f}, Top End Price = {top_price:.2f}")

Year 1: Bottom End Price = 103.00, Top End Price = 108.15
Year 2: Bottom End Price = 105.25, Top End Price = 112.62
Year 3: Bottom End Price = 100.00, Top End Price = 120.00
Year 4: Bottom End Price = 107.50, Top End Price = 118.25
Year 5: Bottom End Price = 109.75, Top End Price = 124.02


In [243]:
# ESG
dir(Ticker('aapl'))


['CHUNK',
 'CORPORATE_EVENTS',
 'FUNDAMENTALS_OPTIONS',
 'FUNDAMENTALS_TIME_ARGS',
 'INTERVALS',
 'MODULES',
 'PERIODS',
 '_CONFIG',
 '_FUND_DETAILS',
 '_MODULES_DICT',
 '_STYLE_BOX',
 '_VIZ_CONFIG',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adjust_ohlc',
 '_async_requests',
 '_chunk_symbols',
 '_construct_data',
 '_construct_params',
 '_construct_urls',
 '_country',
 '_country_params',
 '_financials',
 '_financials_dataframes',
 '_format_data',
 '_fund_holdings',
 '_get_data',
 '_get_symbol',
 '_historical_data_to_dataframe',
 '_history_1m',
 '_option_dataframe',
 '_quote_summary',
 '_quote_summary_dataframe',
 '_setup_url',
 '_symbols',
 '_sy

In [246]:
Ticker('aapl').history()

open        high         low       close     volume  \
symbol date                                                                    
aapl   2024-01-02  187.149994  188.440002  183.889999  185.639999   82488700   
       2024-01-03  184.220001  185.880005  183.429993  184.250000   58414500   
       2024-01-04  182.149994  183.089996  180.880005  181.910004   71983600   
       2024-01-05  181.990005  182.759995  180.169998  181.179993   62303300   
       2024-01-08  182.089996  185.600006  181.500000  185.559998   59144500   
       2024-01-09  183.919998  185.149994  182.729996  185.139999   42841800   
       2024-01-10  184.350006  186.399994  183.919998  186.190002   46792900   
       2024-01-11  186.539993  187.050003  183.619995  185.589996   49128400   
       2024-01-12  186.059998  186.740005  185.190002  185.919998   40444700   
       2024-01-16  182.160004  184.259995  180.929993  183.630005   65603000   
       2024-01-17  181.270004  182.929993  180.300003  182.679993   47317400   
       2024-01-18  186.089996  189.139999  185.830002  188.630005   78005800   
       2024-01-19  189.330002  191.949997  188.820007  191.559998   68741000   
       2024-01-22  192.300003  195.330002  192.259995  193.889999   60133900   
       2024-01-23  195.020004  195.750000  193.830002  195.179993   42355600   
       2024-01-24  195.419998  196.380005  194.339996  194.500000   53631300   
       2024-01-25  195.220001  196.270004  193.110001  194.169998   54822100   
       2024-01-26  194.270004  194.759995  191.940002  192.419998   44594000   
       2024-01-29  192.009995  192.199997  189.580002  191.729996   47145600   
       2024-01-30  190.940002  191.800003  187.470001  188.039993   55859400   
       2024-01-31  187.039993  187.100006  184.350006  184.399994   55467800   
       2024-02-01  183.990005  186.949997  183.820007  186.860001   64885400   
       2024-02-02  179.860001  187.330002  179.250000  185.850006  102518000   
       2024-02-05  188.149994  189.250000  185.839996  187.679993   69668800   
       2024-02-06  186.860001  189.309998  186.770004  189.300003   43490800   
       2024-02-07  190.639999  191.050003  188.610001  189.410004   53439000   
       2024-02-08  189.389999  189.539993  187.350006  188.320007   40962000   
       2024-02-09  188.649994  189.990005  188.000000  188.850006   45155200   
       2024-02-12  188.419998  188.669998  186.789993  187.149994   41781900   
       2024-02-13  185.770004  186.210007  183.509995  185.039993   56529500   
       2024-02-14  185.320007  185.529999  182.440002  184.149994   54630500   
       2024-02-15  183.550003  184.490005  181.350006  183.860001   65434500   
       2024-02-16  183.419998  184.850006  181.669998  182.309998   49701400   
       2024-02-20  181.789993  182.429993  180.000000  181.559998   53665600   
       2024-02-21  181.940002  182.889999  180.660004  182.320007   41529700   
       2024-02-22  183.479996  184.960007  182.460007  184.369995   52292200   
       2024-02-23  185.009995  185.039993  182.229996  182.520004   45119700   
       2024-02-26  182.240005  182.759995  180.649994  181.160004   40867400   
       2024-02-27  181.100006  183.919998  179.559998  182.630005   54318900   
       2024-02-28  182.509995  183.119995  180.130005  181.419998   48953900   
       2024-02-29  181.270004  182.570007  179.529999  180.750000  136682600   
       2024-03-01  179.550003  180.529999  177.380005  179.660004   73488000   
       2024-03-04  176.149994  176.899994  173.789993  175.100006   81510100   
       2024-03-05  170.759995  172.039993  169.619995  170.119995   95132400   
       2024-03-06  171.059998  171.240005  168.679993  169.119995   68587700   
       2024-03-07  169.149994  170.729996  168.490005  169.000000   71765100   
       2024-03-08  169.000000  173.699997  168.940002  170.729996   76114600   

                     adjclose  
symbol date                    
aapl   2024-01-02  185.639999  
       2